In [2]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

2023-06-05 17:25:01.881790: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-05 17:25:01.881811: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-05 17:25:05.206329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-05 17:25:05.206349: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-05 17:25:05.206365: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (amir-asus): /proc/driver/nvidia/version does not exist
2023-06-05 17:25:05.206558: I tensorflow/core/platform/cpu_feature_

### Define function to create scalable hard to sim circuit.

In [11]:
def create_circuit(n_qubits):
    circuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(n_qubits)
    for i in range(n_qubits):
        symbol = sympy.Symbol("Y" + '-' + str(i))
        circuit.append(cirq.Y(qubits[i])**symbol)
    for i in range(n_qubits):
        symbol = sympy.Symbol("XX"+"-"+str(i))
        circuit.append(cirq.XX(qubits[i], qubits[(i+1)%n_qubits])**symbol)
    measurements = []
    for i in range(1):
        measurements.append(cirq.Z(qubits[i]))
    
    return circuit, measurements

quantum_data = tfq.convert_to_tensor([
    cirq.Circuit()
])

## Real tests

In [12]:
steps = 200
file_name="tensorflow_retest.csv"

In [13]:
import time

def test_time(n_qubits):
    circuit, measurements = create_circuit(n_qubits)

    model = tf.keras.Sequential([
        # The input is the data-circuit, encoded as a tf.string
        tf.keras.layers.Input(shape=(), dtype=tf.string),
        # The PQC layer returns the expected value of the readout gate, range [-1,1].
        tfq.layers.PQC(circuit, measurements),
    ])
    
    model.compile(
        loss='mse',
        optimizer=tf.keras.optimizers.Adam())
    beg_steps = 5
    
    model.fit(quantum_data, y=np.ones((1, 1)), 
            epochs=beg_steps,
            batch_size=1,
            verbose=0,)
    
    beg = time.time()
    model.fit(quantum_data, y=np.ones((1, 1)), 
            epochs=steps,
            batch_size=1,
            verbose=0,)
    end = time.time()
    return (end-beg)*1000

In [15]:
from bench_func import run_benchmark

run_benchmark(range(2, 22), test_time, file_name)

  0%|          | 0/20 [00:00<?, ?it/s]

[2, 272.55916595458984]
[3, 345.25156021118164]
[4, 400.4507064819336]
[5, 439.683198928833]
[6, 460.5555534362793]
[7, 520.8158493041992]
[8, 571.3119506835938]
[9, 668.5454845428467]
[10, 715.4850959777832]
[11, 908.7040424346924]
[12, 1116.3244247436523]
[13, 1509.8037719726562]
[14, 2163.4469032287598]
[15, 2763.9291286468506]
[16, 2336.0254764556885]
[17, 3442.8491592407227]
[18, 6251.696825027466]
[19, 12642.646789550781]


KeyboardInterrupt: 

## Testing with qsim from cirq

In [1]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

2023-06-06 16:33:56.362673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 16:33:56.362707: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-06 16:34:00.792920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-06 16:34:00.792941: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-06 16:34:00.792959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (amir-asus): /proc/driver/nvidia/version does not exist
2023-06-06 16:34:00.793556: I tensorflow/core/platform/cpu_feature_

In [16]:
steps = 200
file_name="tensorflow_qsim_16threads.csv"

In [17]:
import time

def test_time(n_qubits):
    circuit, measurements = create_circuit(n_qubits)
    import qsimcirq

    options = {'t': 16}

    qsim_simulator = qsimcirq.QSimSimulator(options)

    model = tf.keras.Sequential([
        # The input is the data-circuit, encoded as a tf.string
        tf.keras.layers.Input(shape=(), dtype=tf.string),
        # The PQC layer returns the expected value of the readout gate, range [-1,1].
        tfq.layers.PQC(circuit, measurements, backend=qsim_simulator),
    ])
    
    model.compile(
        loss='mse',
        optimizer=tf.keras.optimizers.Adam())
    beg_steps = 5
    
    model.fit(quantum_data, y=np.ones((1, 1)), 
            epochs=beg_steps,
            batch_size=1,
            verbose=0,)
    
    beg = time.time()
    model.fit(quantum_data, y=np.ones((1, 1)), 
            epochs=steps,
            batch_size=1,
            verbose=0,)
    end = time.time()
    return (end-beg)*1000

In [18]:
from bench_func import run_benchmark

run_benchmark(range(2, 18), test_time, file_name)

  0%|          | 0/16 [00:00<?, ?it/s]

2023-06-06 16:56:54.903892: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: Adam/gradients/cond/branch_executed/_12


[2, 4408.489465713501]


2023-06-06 16:56:59.864359: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: Adam/gradients/cond/branch_executed/_12


[3, 6498.632192611694]


...

[15, 105170.07637023926]


2023-06-06 17:06:01.340644: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: Adam/gradients/cond/branch_executed/_12


[16, 116665.07649421692]


2023-06-06 17:08:01.336597: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: Adam/gradients/cond/branch_executed/_12


[17, 142045.94492912292]
